# The Google of Quotes (40 points)

 The file “quotes.txt” contains pairs of lines, with the first line being a quoteand  the  following  line  being  the  person  who  said  it.   We  want  to  build  asearch engine that, given a word or words, finds the best matching quotes.

In [1]:
import re
filename = "quotes.txt"
with open(filename,"r") as f:
    content = f.readlines()
    f.close()
print("Number of Lines:", len(content))

Number of Lines: 1772


# Build a list of full quotes (5 points)

Read in the file, and create a list of full quotes of the form “quote - speaker”.

For example,“The hearthas its reasons, of which the mind knows nothing. - Blaise Pascal”.

In [2]:
# Regex to clear string.
# String can contain only Characters, Digits or underscore
content = list(map(lambda x: re.sub('[^A-Za-z0-9_ ]', '', x),content))
content[0:8]

['How we spend our days is of course how we spend our lives',
 'Annie Dillard',
 'Two roads diverged in a wood and II took the one less traveled by and that has made all the difference',
 'Robert Frost',
 'What is happiness The feeling that power is growing that resistance is overcome',
 'Friedrich Nietzsche',
 'A great deal of intelligence can be invested in ignorance when the need for illusion is deep',
 'Saul Bellow']

In [3]:
content = [" - ".join([content[i],content[i+1]]) for i in range(0,len(content),2)]
print(len(content))
content[0:5]

886


['How we spend our days is of course how we spend our lives - Annie Dillard',
 'Two roads diverged in a wood and II took the one less traveled by and that has made all the difference - Robert Frost',
 'What is happiness The feeling that power is growing that resistance is overcome - Friedrich Nietzsche',
 'A great deal of intelligence can be invested in ignorance when the need for illusion is deep - Saul Bellow',
 'Those who are preoccupied with making a statement usually dont have any statements worth making - Thomas Sowell']

# Words from full quotes (5 points)

- Write  a  function  that  takes a full  quote  as  argument  and  outputs  a  list  of  the  words  in  the  it.
- The words should all be lower-case, and should contain only characters, digits, or underscore


In [4]:
def WordSplit(quote: str):
    """
        .
    """
    return tuple(quote.lower().replace(" - "," ").replace("."," ").split())

list(map(WordSplit, content))[0]

('how',
 'we',
 'spend',
 'our',
 'days',
 'is',
 'of',
 'course',
 'how',
 'we',
 'spend',
 'our',
 'lives',
 'annie',
 'dillard')

# Build the postings-list dictionary (6 points)

A postings-list is a dictionary  whose  keys  are  full  quotes,  and  whose  values  are  themselves dictionaries  with  key  being  a  word,  and  value  being  the  number  of  times the  word  occurs  in  the  full  quote.

So, for the **key** _"The heart has its reasons, of which the mind knows nothing. - Blaise Pascal"_,

the value will itself be the following  dictionary:

```Python
{"the":2, "heart":1, "has":1, "its":1, "reasons":1, "of":1, "which":1, "mind":1, "knows":1, "nothing":1, "blaise":1, "pascal":1}
```


In [5]:
def TfDict(words: tuple):
    """
        .
    """
    tfDict = dict()
    for word in words:
        if word not in tfDict.keys():
            tfDict[word] = 1
        else:
            tfDict[word] += 1
    return tfDict

list(map(TfDict,map(WordSplit, content)))[0]

{'how': 2,
 'we': 2,
 'spend': 2,
 'our': 2,
 'days': 1,
 'is': 1,
 'of': 1,
 'course': 1,
 'lives': 1,
 'annie': 1,
 'dillard': 1}

In [6]:
postingsList = dict()
for quote in content:
    if quote not in postingsList.keys():
        postingsList[quote] = TfDict(WordSplit(quote))
    else:
        print("Duplicate Quote:\n{quote}".format(quote = quote))
postingsList['How we spend our days is of course how we spend our lives - Annie Dillard']

{'how': 2,
 'we': 2,
 'spend': 2,
 'our': 2,
 'days': 1,
 'is': 1,
 'of': 1,
 'course': 1,
 'lives': 1,
 'annie': 1,
 'dillard': 1}

# Build the reverse postings-list dictionary (6 points)

A reverse postings-list is a dictionary whose keys are the words, and the values are themselves dictionaries with the key being a full quote, and the value being the number of times the word appeared in the full quote.

So, for the key _"entertainer"_, the value is the dictionary

```Python
{"An actor is at most a poet and at least an entertainer. - Marlon Brando":  1} 
#only this quote contains the word “entertainer”
```


In [7]:
# Create Set of all words
allWords = set(" ".join(content).lower().replace(" - "," ").replace("."," ").split())
print(len(allWords))

3888


In [8]:
def IdfDict(word: str):
    """
        .
    """
    idfDict = dict()
    for quote in content:
        if word in postingsList[quote].keys():
            idfDict[quote] = postingsList[quote][word]
    return idfDict

IdfDict("spend")

{'How we spend our days is of course how we spend our lives - Annie Dillard': 2,
 'Are you entitled to the fruits of your labor or does government have some presumptive right to spend and spend and spend - Ronald Reagan': 3,
 'If God had wanted us to spend our time fretting about the problems of home ownership He would never have invented beer - Dave Barry': 1}

In [9]:
revPostingsList = dict()
for word in allWords:
    if word not in revPostingsList.keys():
        revPostingsList[word] = IdfDict(word)
    else:
        print("Duplicate Word:\n{word}".format(word = word))
revPostingsList["entertainer"]

{'An actor is at most a poet and at least an entertainer - Marlon Brando': 1}

# Write a TF-IDF function (8 points)

To measure how much a full quote is about a particular word, one typically uses the TF-IDF measure

## TF

- TF  stands  for  **term  frequency**
- The  term  frequency  of  a  word _w_ in a  full  quote _q_ is  the  **number  of  times** _w_ occurs  in q,  divided  by  the maximum number of times any word occurs in _q_

## IDF

- IDF stands for **inverse document frequency**
- The IDF of a word _w_ is the logarithm of the ratio of the total number _N_ of full quotes to the number of full quotes in that contain the word _w_

TF-IDF of a word _w_ for a full quote _q_ is just the **product** of the **TF** and **IDF**.

So, for the word _entertainer_ in the Marlon Brando quote of part (d):

- The  TF  is  0.5  (it  occurs  once,  while  the  most  frequent  word  in  that quote is “at”, which occurs twice, so the TF ratio is 0.5)
- The IDF is log (886/1),  since there are 886 documents and the word“entertainer ”occurs in only one full quote.

Write a function to compute the TF-IDF of any word in any full quote, using the postings and reverse-postings.

Hint:Do import math and use _math.log()_ to get logarithms.


In [10]:
import operator
def MaxWordFreqinQuote(quote: str):
    return max(postingsList[quote].items(), key=operator.itemgetter(1))[1]

In [11]:
import math
def TfIdf(word: str, quote: str):
    """
        .
    """
    tf, idf = 0,0

    # Check if Quote Exists and Word exists in Quote. Else, return 0 (b/c TF = 0)
    if (quote in postingsList.keys())and (word in postingsList[quote].keys()):
        tf = postingsList[quote][word]/MaxWordFreqinQuote(quote)
    
    # Check if Word exists in revPostingList. Else, return 0 (b/c IDF = 0)
    if word in revPostingsList.keys():
        idf = math.log(len(content) / len(revPostingsList[word].keys()))
    else:
        return "Word Does not exist"
    # print("TF: {tfscore};\nIDF: log({N}/{x}) = {idfscore}".format(tfscore = tf,
    #                                                             idfscore = idf,
    #                                                             N = len(content),
    #                                                             x = len(revPostingsList[word].keys())))
    return tf*idf

## Quote  search  using  a  single  word  (5  points)

Write  a  function that takes a word as argument, and returns a dictionary whose keys are full quotes containing that word, and whose values are the TF-IDF score of thatword for that full quote.


In [12]:
def SingleWordScore(word: str):
    """
        Input: Word 
        Returns: A dictionary:   KEYS -> full quotes containing that word
                                VALUE -> TF-IDF score of the word for that full quote
    """
    score = dict()
    if word in revPostingsList.keys():
        for quote in revPostingsList[word]:
            score[quote] = TfIdf(word = word, quote= quote)
    return score

SingleWordScore("spent")

{'I feel impelled to speak today in a language that in a sense is newone which I who have spent so much of my life in the military profession would have preferred never to use That new language is the language of atomic warfare - Dwight Eisenhower': 2.0311899233483786,
 'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust': 3.046784885022568}

In [13]:
SingleWordScore("book")

{'The Bible is not my book and Christianity is not my religion I could never give assent to the long complicated statements of Christian dogma - Abraham Lincoln': 2.497478740688513,
 'There is a great deal of difference between an eager man who wants to read a book and the tired man who wants a book to read - GK Chesterton': 3.3299716542513504,
 'The covers of this book are too far apart - Ambrose Bierce': 4.994957481377026,
 'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust': 2.497478740688513,
 'A truly great book should be read in youth again in maturity and once more in old age as a fine building should be seen by morning light at noon and by moonlight - Robertson Davies': 1.6649858271256752,
 'Dont join the book burners Do not think you are going to conceal thoughts by concealing evidence that they ever existed - Dwight Eisenhower': 4.994957481377026}

## Quote search using multiple words (5 points)

Write a function that takes a list of words as argument, and returns a dictionary whose keys are full quotes containing one or more of the words in that list, and whose values are the **sum of TF-IDF scores** of the words in that list for that full-quote.


In [14]:
def MultiWordScore(wordList: list):
    """
        Input: List of Words
        Returns: A dictionary:   KEYS -> full quotes containing that word(s)
                                VALUE -> sum of TF-IDF score of the word(s) for that full quote
    """
    score = dict()
    for word in wordList:
        if word in revPostingsList.keys():
            for quote in revPostingsList[word]:
                if quote in score.keys():
                    score[quote] += TfIdf(word = word, quote= quote)
                else:
                    score[quote] = TfIdf(word = word, quote= quote)
    return score

MultiWordScore(["spent","book","entertainer"])

{'I feel impelled to speak today in a language that in a sense is newone which I who have spent so much of my life in the military profession would have preferred never to use That new language is the language of atomic warfare - Dwight Eisenhower': 2.0311899233483786,
 'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust': 5.54426362571108,
 'The Bible is not my book and Christianity is not my religion I could never give assent to the long complicated statements of Christian dogma - Abraham Lincoln': 2.497478740688513,
 'There is a great deal of difference between an eager man who wants to read a book and the tired man who wants a book to read - GK Chesterton': 3.3299716542513504,
 'The covers of this book are too far apart - Ambrose Bierce': 4.994957481377026,
 'A truly great book should be read in youth again in maturity and once more in old age as a fine building should be seen by morning light at noon and by moonlight

<br><p>
**TF-IDF** score of _**spent**_ in _'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust'_ : **3.0467**
<br><p>
**TF-IDF** score of _**book**_ in _'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust'_ : **2.4974**
<br><p>
**TF-IDF** score of _**spent, book**_ in _'There are perhaps no days of our childhood we lived so fully as those we spent with a favorite book - Marcel Proust'_ : **5.54426**